In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import seaborn as sns
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
#from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2train.csv", header=0) 
df.head()

In [ ]:
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(df)

In [ ]:
df["has_colection"] = df['belongs_to_collection'].apply(lambda x: lambda: len(x) if x != {} else 0)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#df = df.dropna()
#df['belongs_to_collection'] = df['belongs_to_collection'].fillna(0)
df = df.drop(["homepage",'belongs_to_collection' ], axis = 1)

In [ ]:
df.info()

In [ ]:
df.dropna()

In [ ]:
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [ ]:
Y = df["revenue"]

In [ ]:
df.head()

In [ ]:
Y.head()

In [ ]:
def has_collection(column):
    if "id" in column: 
        return 1
    else: 
        return 0

In [ ]:
#df["IsCollection"] = df["belongs_to_collection"].apply(has_collection)

In [ ]:
#df["IsCollection"].value_counts()

In [ ]:
df.head()

In [ ]:
sns.scatterplot(df['popularity'],df['revenue'],color='blue')
plt.show()

In [ ]:
sns.scatterplot(df['budget'],df['revenue'],color='blue')

In [ ]:
sns.scatterplot(df['runtime'],df['revenue'],color='blue')

In [ ]:
df["budget_year_ratio"] = df['budget'] / (df['year'] * df['year'])

In [ ]:
#cast = df[["num_cast"]]

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
X = df[["popularity", "runtime", "budget", "budget_year_ratio", "year", ]]

In [ ]:
y = df[["revenue"]]

In [ ]:
y = 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)


In [ ]:
xgbr.fit(X_train, y_train)

In [ ]:
ypred = xgbr.predict(X_test)
mse = mean_squared_error(y_test, ypred)
print("MSE: %.2f" % mse)
MSE: 3.35
print("RMSE: %.2f" % (mse**(1/2.0)))
RMSE: 1.83 